In [1]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
def angle_between(v1:npt.ArrayLike, v2:npt.ArrayLike) -> float:
    """Angle calculation between two vectors is performed as follows:
    
    1. Calculate dot product v1 * v2
    2. Divide it by multiplication of magnitudes of v1 and v2.
    3. Put the result in arccos function.
    
    In other words, it is a transformed dot product equation.

    Return: Angle in radians (to obtain degrees multipliy it by: 180/pi)
    """
    
    return np.arccos(np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))


In [3]:
def calc_3d_vector(ptA:npt.ArrayLike, ptB:npt.ArrayLike) -> npt.ArrayLike:
    if len(ptA) != 3 or len(ptB) != 3:
        raise ValueError("Given point(s) are not in three dimensional space.")
    return np.subtract(ptB,ptA)
    

In [4]:
def calc_angle(ptA:pd.Series, ptB:pd.Series, ptC:pd.Series, ptD:pd.Series) -> float:
    v1 = calc_3d_vector(ptA.values, ptB.values)
    v2 = calc_3d_vector(ptC.values, ptD.values)
    # import vg
    # return vg.angle(v1, v2)
    return angle_between(v1,v2)*180/np.pi
    

In [5]:
def calc_point_speed(colx:pd.Series, coly:pd.Series, colz:pd.Series, time:pd.Series):
    p1 = np.array([colx, coly, colz])
    p2 = np.array([colx.shift(1), coly.shift(1), colz.shift(1)])

    squared_dist = np.sum((p1-p2)**2, axis=0)
    dist = np.sqrt(squared_dist)
    return dist/time
    

In [6]:
def calc_point_distance_from_floor(
        colx:pd.Series, coly:pd.Series, colz:pd.Series, floorx:pd.Series, floory:pd.Series, floorz:pd.Series, floorw:pd.Series):
    numerator = colx*floorx + coly*floory + colz*floorz + floorw
    denominator = np.sqrt(floorx*floorx + floory*floory + floorz*floorz)
    return numerator/denominator
    

In [13]:
plt.close("all")
df = pd.read_csv('data\\setup_mid\\good\\20231106_1224\\trimmed - 20231106122417713.csv')
df = df[["Time",
                  "KneeRight_x",  "KneeRight_y",  "KneeRight_z",
                  "HipRight_x",   "HipRight_y",   "HipRight_z",
                  "KneeLeft_x",   "KneeLeft_y",   "KneeLeft_z",   
                  "HipLeft_x",    "HipLeft_y",    "HipLeft_z",
                  "FootRight_x",  "FootRight_y",  "FootRight_z",
                  "Floor_x",  "Floor_y",  "Floor_z", "Floor_w"]]
df.head()

,Time,KneeRight_x,KneeRight_y,KneeRight_z,HipRight_x,HipRight_y,HipRight_z,KneeLeft_x,KneeLeft_y,KneeLeft_z,HipLeft_x,HipLeft_y,HipLeft_z,FootRight_x,FootRight_y,FootRight_z,Floor_x,Floor_y,Floor_z,Floor_w
0,4.861265,0.049008,-0.256081,1.916973,-0.005131,-0.024597,2.256264,-0.197190,-0.326051,1.935639,-0.142482,-0.040834,2.251824,0.077915,-0.839378,1.986076,-0.005029,0.999943,0.009371,0.943502
1,4.894327,0.053471,-0.221627,1.940372,-0.000055,0.050242,2.270799,-0.197448,-0.288252,1.964580,-0.132855,0.021013,2.253197,0.078069,-0.804415,1.958859,-0.005029,0.999943,0.009371,0.943502
2,4.927398,0.096337,-0.145795,1.909148,-0.007436,0.079105,2.264136,-0.208361,-0.295638,2.017953,-0.142040,0.055723,2.245056,-0.010338,-0.748535,1.912109,-0.005029,0.999943,0.009371,0.943502
3,4.960957,0.086850,0.002267,1.948403,-0.102952,0.210096,2.276500,-0.254164,-0.207932,2.066446,-0.240723,0.181216,2.222936,-0.044562,-0.562433,1.929283,-0.005029,0.999943,0.009371,0.943502
4,4.993513,0.084261,0.067937,1.941179,-0.119176,0.277158,2.259958,-0.195861,-0.085361,2.042155,-0.245747,0.270748,2.222786,0.029168,-0.476566,1.879316,-0.005029,0.999943,0.009371,0.943502


In [11]:
df["crotch_angle"] = df.apply(
    lambda row: calc_angle(row[1:4], row[4:7], row[7:10], row[10:13]), axis=1)

df['crotch_angle_smooth'] = signal.savgol_filter(df['crotch_angle'], window_length=11, polyorder=3, mode="nearest")

df['foot_speed'] = calc_point_speed(df.FootRight_x, df.FootRight_y, df.FootRight_z, df.Time)

df['foot_speed_smooth'] = signal.savgol_filter(df['foot_speed'], window_length=6, polyorder=3, mode="nearest")

df['foot_floor_distance'] = calc_point_distance_from_floor(
    df.FootRight_x, df.FootRight_y, df.FootRight_z, df.Floor_x, df.Floor_y, df.Floor_z, df.Floor_w)

df['foot_floor_distance_smooth'] = signal.savgol_filter(df['foot_floor_distance'], window_length=11, polyorder=3, mode="nearest")

In [14]:
ax_crotch = df.plot(kind="line", x="Time", y="crotch_angle_smooth", label="Smoothened crotch angle [deg]")
df.plot(kind="line", x="Time", y="crotch_angle", label="Crotch angle [deg]", 
        title="Crotch angle over time while performing Ollie", ax=ax_crotch)

ax_foot = df.plot(kind="line", x="Time", y="foot_speed_smooth", label="Smoothened foot speed [unit/s]")
df.plot(kind="line", x="Time", y="foot_speed", label="Foot speed [unit/s]", 
        title="Foot speed over time while performing Ollie", ax=ax_foot)

ax_floor_dist = df.plot(kind="line", x="Time", y="foot_floor_distance_smooth", label="Smoothened foot-floor distance [unit]")
df.plot(kind="line", x="Time", y="foot_floor_distance", label="Foot-floor distance [unit]", 
        title="Foot-floor distance over time while performing Ollie", ax=ax_floor_dist)

plt.show()

KeyError: 'crotch_angle_smooth'